In [2]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import networkx as nx
import json

In [3]:
df_C_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="CDIO_COO_DIV PRIVATE BANKING")
df_P_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Private Banking")
df_OP = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Organizational Position")

df_Employees = pd.read_excel("Data.xlsx", sheet_name="Employees")
df_Domain = pd.read_excel("Data.xlsx", sheet_name="Knowledge structure")
df_Assessment = pd.read_excel("Data.xlsx", sheet_name="Knowledge assessment", na_values = "NA")
df_Organization = pd.read_excel("Data.xlsx", sheet_name="Organizational structure", na_values="NA")
df_Persona = pd.read_excel("Data.xlsx", sheet_name="Personas")

## Employment

### Distribution of Age

In [4]:
df_Employees['AGE'].describe() 

count    65021.000000
mean        49.680273
std          8.618742
min         21.000000
25%         43.000000
50%         51.000000
75%         57.000000
max         71.000000
Name: AGE, dtype: float64

### Counts for Sex

In [5]:
df_Employees['SEX'].value_counts()

F    33016
M    31691
Name: SEX, dtype: int64

### Count Employee

In [6]:
len(df_Employees['ID_EMPLOYEE'].unique())

64998

In [7]:
len(df_Employees.index)

65021

### There should be duplicated ID_EMPLOYEE

In [10]:
a_list = df_Employees.ID_EMPLOYEE.tolist()
a_set = set(a_list)
contains_duplicates = len(a_list) != len(a_set)

list1=[]
duplicated_ID = []
for i in a_list:
    if i not in list1:
        list1.append(i)
    else:
        duplicated_ID.append(i)
len(duplicated_ID)

23

### Duplicated ID have different Orgnization_Position

In [11]:
df_Employees.loc[df_Employees['ID_EMPLOYEE'].isin(duplicated_ID)]

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION
8421,8503,M,62,QD/4,#,4593
8422,8503,M,62,QD/4,#,5051
9955,10049,M,57,QD/4,#,4596
9956,10049,M,57,QD/4,#,5107
11006,11113,M,51,QD/2,#,4593
11007,11113,M,51,QD/2,#,5051
29772,30411,M,63,QD/1,#,4607
29773,30411,M,63,QD/1,#,6145
32405,33046,M,63,QD/4,#,4607
32406,33046,M,63,QD/4,#,6145


## Orgnizational Structure

In [94]:
len(df_Organization['ID_PERSONAS'].unique())

37

## Knowledge Structure 

In [31]:
df_Domain.loc[df_Domain["DESCR_KNOWLEDGE_DOMAIN"].isnull()]

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA


#### Two Knowledge domains don't have associated Knowlege Area

In [32]:
df_Domain.loc[df_Domain["DESCR_KNOWLEDGE_AREA"].isnull()]

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
49,C01D03,Segmento Imprese,NaN,NaN
85,C32D06,Relazioni con i Regolatori,NaN,NaN


In [28]:
len(df_Domain['ID_KNOWLEDGE_DOMAIN'].unique())
# no duplicated index
len(df_Domain['ID_KNOWLEDGE_DOMAIN'].unique()) == len(df_Domain.index)

True

#### 130 KNOWLEDGE_DOMAIN, 43 KNOWLEDGE_AREA

In [29]:
len(df_Domain['ID_KNOWLEDGE_DOMAIN'].unique())

130

In [161]:
len(df_Domain['ID_KNOWLEDGE_AREA'].unique())

43

In [30]:
df_Domain['ID_KNOWLEDGE_AREA'].value_counts()

AREAC016_1    15
AREAC028       6
AREAC003       6
AREAC030_1     5
AREAC005_1     5
AREAC006       4
AREAC030_2     4
AREAC032       4
AREAC034_1     4
AREAC004       3
AREAC025       3
AREAC017       3
AREAC018       3
AREAC015       3
AREAC001       3
AREAC012       3
AREAC033_1     3
AREAC007       3
AREAC002       3
AREAC026       3
AREAC027       3
AREAC011       3
AREAC035_1     3
AREAC019       2
AREAC036       2
AREAC035_2     2
AREAC019_1     2
AREAC023       2
AREAC022       2
AREAC041       2
AREAC013       2
AREAC029       2
AREAC038       2
AREAC008       2
AREAC014       2
AREAC010       2
AREAC024       2
AREAC009       1
AREAC016_2     1
AREAC040       1
AREAC039       1
AREAC037       1
Name: ID_KNOWLEDGE_AREA, dtype: int64

## Assessment

In [148]:
df_Assessment

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
0,63374,767,C05_1D01,1,NaN
1,39838,42166,C11D02,1,NaN
2,27198,8078,C15D03,1,1.0
3,28622,20863,C23D02,1,NaN
4,20494,25046,C28D04,1,NaN
...,...,...,...,...,...
1048570,34556,35036,C35_1D03,1,NaN
1048571,265,18190,C07D01,1,NaN
1048572,3117,67408,C41D01,1,NaN
1048573,63649,63704,C32D03,1,1.0


### distribution of SCORE_AUTODETECTION

In [151]:
df_Assessment['SCORE_AUTODETECTION'].value_counts()

1    864221
2    113101
3     46071
4     19264
5      5918
Name: SCORE_AUTODETECTION, dtype: int64

### A large number of Employees don't have SCORE_VALIDATION in specific Knowledge domain. Some knowledge domains have large missing self-score. 98% of the employees have at least one missing score validation in a knowledege domain. 

In [22]:
df_ValidationScore= df_Assessment.loc[df_Assessment['SCORE_VALIDATION'].isna()]
df_ValidationScore['ID_KNOWLEDGE_DOMAIN'].value_counts()
len(df_ValidationScore['ID_EMPLOYEE'].unique()) / len(df_Assessment['ID_EMPLOYEE'].unique())

0.9831103988164951

In [23]:
df_With_Validation = df_Assessment[df_Assessment['SCORE_VALIDATION'].notna()]

In [24]:
df_With_Validation

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
2,27198,8078,C15D03,1,1.0
11,54759,43553,C29D02,1,1.0
16,8217,3976,C11D03,1,1.0
20,26640,28488,C41D01,1,1.0
26,44864,4853,C30_1D05,1,1.0
...,...,...,...,...,...
1048566,17752,17858,C01D04,1,1.0
1048567,44217,1903,C03D01,5,5.0
1048569,16261,63573,C16_1D13,1,1.0
1048573,63649,63704,C32D03,1,1.0


### 86.9 percent of self score match the manager score. 

In [156]:
(df_With_Self_Score['SCORE_AUTODETECTION'] - df_With_Self_Score['SCORE_VALIDATION']).value_counts(normalize = True)

 0.0    0.868949
 1.0    0.050125
-1.0    0.046603
 2.0    0.013240
-2.0    0.010800
 3.0    0.005146
-3.0    0.002656
 4.0    0.002017
-4.0    0.000463
dtype: float64

### Create Network Data

In [36]:
df_C_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="CDIO_COO_DIV PRIVATE BANKING")
df_P_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Private Banking")

In [37]:
df_C = df_C_raw[["UOG1_F","UOG2_F","UOG3_F","UOG4_F","UOG5_F","UOG6_F"]]
df_P = df_P_raw[["UOG1_F","UOG2_F","UOG3_F","UOG4_F","UOG5_F","UOG6_F"]]

In [38]:
# extract all org id
org_list = []
for i in df_C:
    org_list.append(df_C[i].tolist())

for i in df_P:
    org_list.append(df_P[i].tolist())

# flattern list
flat_list = []
for sublist in org_list:
    for item in sublist:
        flat_list.append(item)
        
org_id = np.unique(np.array(flat_list)).tolist()
org_id.remove("-")
org_id = list(map(int, org_id))

#### Extract all the unique orgnization_positions(from Private bankings) intended to investigate from the whole employees data

In [39]:
org_id

[1,
 10,
 100,
 10012,
 10019,
 10020,
 10036,
 10048,
 10049,
 10056,
 10077,
 10079,
 101,
 10112,
 10113,
 10115,
 10119,
 10120,
 102,
 103,
 104,
 105,
 10571,
 10572,
 10573,
 10574,
 10575,
 10576,
 10577,
 10578,
 10579,
 10580,
 10581,
 10582,
 10583,
 10584,
 10585,
 106,
 107,
 10730,
 10731,
 10732,
 10733,
 10734,
 10735,
 10736,
 10737,
 10738,
 10739,
 10740,
 10741,
 10742,
 10743,
 10744,
 10745,
 10746,
 10747,
 10748,
 10749,
 10750,
 10751,
 10752,
 10753,
 10754,
 10755,
 10756,
 10757,
 10758,
 10759,
 10760,
 10761,
 10762,
 10763,
 10764,
 10765,
 10766,
 10767,
 10768,
 10769,
 10770,
 10771,
 10772,
 10773,
 10774,
 10775,
 10776,
 10777,
 10778,
 10779,
 10780,
 10781,
 10782,
 10783,
 10784,
 10785,
 10786,
 10787,
 10788,
 10789,
 10790,
 10791,
 10792,
 10793,
 10794,
 10795,
 10796,
 10797,
 10798,
 10799,
 108,
 10800,
 10801,
 10802,
 10803,
 10804,
 10805,
 10806,
 10807,
 10808,
 10809,
 10810,
 10811,
 10812,
 10813,
 10814,
 10815,
 10816,
 10817,
 

### Filter the Employees that have the orgnization in the dataset

In [72]:
# filter employee ID by org ID
df_name = pd.merge(df_Employees, df_Organization, left_on = "ID_ORGANIZATIONAL_POSITION", 
                   right_on = "ID_ORGANIZATIONAL_POSITION")
df_name = df_name.loc[df_name["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]

#### No more duplicated EmployeeID : one employee only has one orgnizational position

In [73]:
len(df_name['ID_EMPLOYEE'].unique()) == len(df_name.index)

True

In [74]:
df_assess = pd.merge(df_Assessment, df_Domain, left_on = "ID_KNOWLEDGE_DOMAIN", 
                         right_on = "ID_KNOWLEDGE_DOMAIN")

In [77]:
# filter employee assessment by org ID
df = pd.merge(df_name, df_assess, left_on = "ID_EMPLOYEE",
             right_on = "ID_EMPLOYEE")
df = df.loc[df["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]


In [78]:
df

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
0,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C05_1D01,1,1.0,Prodotti danni,AREAC005_1,Conoscenza di Prodotti e Servizi Assicurativi
1,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C11D01,1,1.0,Istruttoria e concessione del credito,AREAC011,Analisi e Gestione del credito
2,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C34_1D03,1,NaN,Gestione servizi a supporto,AREAC034_1,Property
3,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C12D01,4,NaN,Sistema finanziario,AREAC012,Capital Markets
4,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C23D01,1,NaN,Analisi statistiche,AREAC023,Modelli quantitativi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160730,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C30_1D01,1,NaN,Communication Plan,AREAC030_1,Comunicazione
160731,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C06D03,1,NaN,Servizi professionali alle imprese,AREAC006,Conoscenza di Prodotti e Servizi non bancari
160732,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C38D01,1,1.0,Program & Project Management,AREAC038,Program & Project Management
160733,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C04D02,1,1.0,Trade Finance,AREAC004,Transaction Banking


In [79]:
duplicate = df[df.duplicated()]
len(duplicate)

31

In [80]:
duplicate

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
1086,10863,F,61,DR/0,X,8275,Milano,Lombardia,P00019,776,C02D03,1,1.0,Customer Development,AREAC002,Customer Relationship & Development
28602,15727,M,48,DR/0,X,8239,Milano,Lombardia,P00019,776,C16_1D06,1,NaN,Normativa specialistica,AREAC016_1,Normativa e Compliance
39171,13862,M,48,QD/4,X,8247,Milano,Lombardia,P00019,15223,C28D04,1,NaN,Sistemi retributivi,AREAC028,Risorse Umane
39183,13862,M,48,QD/4,X,8247,Milano,Lombardia,P00019,15223,C36D02,1,1.0,Service Design,AREAC036,Digital
52554,5425,M,49,DR/0,X,8289,Milano,Lombardia,P00019,15545,C41D01,1,1.0,Relazioni internazionali,AREAC041,International Network
64703,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C29D02,2,NaN,Processi e procedure aziendali,AREAC029,Modelli e processi organizzativi
64714,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C26D03,1,NaN,Gestione e manutenzione IT,AREAC026,Architettura IT e software development
64722,13783,M,63,DR/0,X,8287,Parma,Emilia Romagna,P00019,15093,C22D02,2,2.0,Data Analysis,AREAC022,Data governance
82040,13428,M,59,QD/4,X,8292,Padova,Veneto,P00019,62825,C14D02,1,NaN,ALM & Capital Management,AREAC014,Funding
82377,13584,F,47,QD/4,X,8277,Parma,Emilia Romagna,P00019,14847,C28D03,1,NaN,Gestione e Valutazione,AREAC028,Risorse Umane


### drop the duplicated rows

In [135]:
df_no_duplicate = df.drop_duplicates( keep='last')
df_no_duplicate

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
0,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C05_1D01,1,1.0,Prodotti danni,AREAC005_1,Conoscenza di Prodotti e Servizi Assicurativi
1,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C11D01,1,1.0,Istruttoria e concessione del credito,AREAC011,Analisi e Gestione del credito
2,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C34_1D03,1,NaN,Gestione servizi a supporto,AREAC034_1,Property
3,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C12D01,4,NaN,Sistema finanziario,AREAC012,Capital Markets
4,6,F,52,QD/4,X,55,Milano,Lombardia,P00030,13080,C23D01,1,NaN,Analisi statistiche,AREAC023,Modelli quantitativi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160730,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C30_1D01,1,NaN,Communication Plan,AREAC030_1,Comunicazione
160731,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C06D03,1,NaN,Servizi professionali alle imprese,AREAC006,Conoscenza di Prodotti e Servizi non bancari
160732,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C38D01,1,1.0,Program & Project Management,AREAC038,Program & Project Management
160733,68363,M,49,DR/0,X,10731,Milano,Lombardia,P00036,14353,C04D02,1,1.0,Trade Finance,AREAC004,Transaction Banking


## Analyze Knowledge Area: using df_no_duplicate

In [161]:
# All possible Knowledge areas are included
len(df_no_duplicate['ID_KNOWLEDGE_AREA'].unique())

43

#### No knowledge domain has NaN values meaning that there is at list one SCORE_VALIDATION for each domain from any employee. The SCORE_AUTODETECTION for each knowledge shows a small difference, not significant to compare anverage scores between each domain. May need to rescale. 

In [129]:
df_Knowledge_area_domain = df_no_duplicate.groupby(["DESCR_KNOWLEDGE_AREA","DESCR_KNOWLEDGE_DOMAIN"]).mean()
df_Knowledge_area_domain.drop(["ID_EMPLOYEE","ID_ORGANIZATIONAL_POSITION","ID_MANAGER"], axis=1)

AGE  \
DESCR_KNOWLEDGE_AREA      DESCR_KNOWLEDGE_DOMAIN                                          
Advisory                  Advisory e negoziazione M&A                         49.543333   
                          Finanza strutturata                                 49.944306   
Amministrazione           Analisi e redazione bilancio                        49.706838   
                          Contabilit√† generale (nazionale e internazionale)  49.949640   
                          Fiscalit√† d'impresa                                50.218960   
...                                                                                 ...   
Transaction Banking       Cash Management                                     49.632164   
                          Securities Services                                 49.307313   
                          Trade Finance                                       49.641618   
Wealth & Asset Management Asset management                                    49.700627   
                          Wealth management                                   49.736000   

                                                                              SCORE_AUTODETECTION  \
DESCR_KNOWLEDGE_AREA      DESCR_KNOWLEDGE_DOMAIN                                                    
Advisory                  Advisory e negoziazione M&A                                    1.081667   
                          Finanza strutturata                                            1.062344   
Amministrazione           Analisi e redazione bilancio                                   1.109402   
                          Contabilit√† generale (nazionale e internazionale)             1.126299   
                          Fiscalit√† d'impresa                                           1.065436   
...                                                                                           ...   
Transaction Banking       Cash Management                                                1.110351   
                          Securities Services                                            1.201315   
                          Trade Finance                                                  1.119736   
Wealth & Asset Management Asset management                                               1.254702   
                          Wealth management                                              1.297600   

                                                                              SCORE_VALIDATION  
DESCR_KNOWLEDGE_AREA      DESCR_KNOWLEDGE_DOMAIN                                                
Advisory                  Advisory e negoziazione M&A                                 1.069061  
                          Finanza strutturata                                         1.042289  
Amministrazione           Analisi e redazione bilancio                                1.144385  
                          Contabilit√† generale (nazionale e internazionale)          1.196970  
                          Fiscalit√† d'impresa                                        1.046729  
...                                                                                        ...  
Transaction Banking       Cash Management                                             1.137079  
                          Securities Services                                         1.194503  
                          Trade Finance                                               1.180495  
Wealth & Asset Management Asset management                                            1.170874  
                          Wealth management                                           1.261278  

[128 rows x 3 columns]

In [132]:
df_Knowledge_area_domain.loc[df_Knowledge_area_domain['SCORE_VALIDATION'].isna()]

,,ID_EMPLOYEE,AGE,ID_ORGANIZATIONAL_POSITION,ID_MANAGER,SCORE_AUTODETECTION,SCORE_VALIDATION
DESCR_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_DOMAIN,,,,,,


In [130]:
df_Knowledge_area = df_no_duplicate.groupby(["DESCR_KNOWLEDGE_AREA"]).mean()
df_Knowledge_area.drop(["ID_EMPLOYEE","ID_ORGANIZATIONAL_POSITION","ID_MANAGER"], axis=1)

,AGE,SCORE_AUTODETECTION,SCORE_VALIDATION
DESCR_KNOWLEDGE_AREA,,,
Advisory,49.744070,1.071993,1.054974
Amministrazione,49.959867,1.100747,1.122792
Analisi e Gestione del credito,49.714973,1.110160,1.084265
Analisi economica e finanziaria,49.482526,1.146942,1.081633
Architettura IT e software development,49.843386,1.374339,1.419530
Business Development,49.795943,1.222753,1.175658
Capital Markets,49.676985,1.204307,1.208993
Change Management,49.625620,1.221488,1.167488
Comunicazione,49.604053,1.139273,1.117035


## Analyze Personas: using df_no_duplicate

### Employee dataset include 27 personas. 37 personas could cover all possible knowledge areas

In [136]:
df_persona = df_no_duplicate.drop_duplicates(subset=['ID_EMPLOYEE'],keep='first')
# personals presenting in employee dataset
persona_list = df_persona['ID_PERSONAS'].unique().tolist()
len(persona_list)

27

### The number of employees included in each Persona varies significantly

In [165]:
df_no_duplicate.drop_duplicates(subset = ['ID_EMPLOYEE'],keep='first')['ID_PERSONAS'].value_counts()

P00019    3343
P00014    2099
P00029    1744
P00004     574
P00005     294
P00006     242
P00016     222
P00013     163
P00015     151
P00020     150
P00030     113
P00007     110
P00018     106
P00008      78
P00036      70
P00023      66
P00025      61
P00009      56
P00001      48
P00024      42
P00035      41
P00002      40
P00010      40
P00026      31
P00034      27
P00032       8
P00003       2
Name: ID_PERSONAS, dtype: int64

### If we use one persona as a subset, 26 out of 27 personas subset of employees would associate with all the knowledge areas. Even though numbers of employees in each persona is significantly different, one persona could capture all knowledge areas.

In [192]:
persona_list=df_no_duplicate['ID_PERSONAS'].unique().tolist()
df_persona_group = df_no_duplicate[['ID_PERSONAS','DESCR_KNOWLEDGE_AREA']].groupby('ID_PERSONAS')

for i in persona_list:
    print(i, len(df_persona_group.get_group(i)['DESCR_KNOWLEDGE_AREA'].unique()))

P00030 43
P00019 43
P00029 43
P00004 43
P00010 43
P00007 43
P00032 38
P00005 43
P00035 43
P00014 43
P00026 43
P00020 43
P00008 43
P00023 43
P00016 43
P00001 43
P00006 43
P00036 43
P00025 43
P00018 43
P00002 43
P00015 43
P00009 43
P00013 43
P00024 43
P00034 43
P00003 24


### For each Persona, there may be some DESCR_KNOWLEDGE_AREA has no employee has SCORE_VALIDATION, (showing NaN). If use Persona as filter, the Kowledge area without any SCORE_VALIDATION may be missleading. 

In [188]:
df_persona_score = df_no_duplicate.groupby(["ID_PERSONAS","DESCR_KNOWLEDGE_AREA"]).mean()
df_persona_score.drop(["ID_EMPLOYEE","ID_ORGANIZATIONAL_POSITION","ID_MANAGER"], axis=1)
df_persona_score

ID_EMPLOYEE        AGE  \
ID_PERSONAS DESCR_KNOWLEDGE_AREA                                              
P00001      Advisory                                24121.923077  51.153846   
            Amministrazione                         41301.000000  53.444444   
            Analisi e Gestione del credito          28991.857143  52.500000   
            Analisi economica e finanziaria         36235.307692  56.615385   
            Architettura IT e software development  19308.058824  55.823529   
...                                                          ...        ...   
P00036      Risorse Umane                           44288.508772  48.789474   
            Safety                                  50452.888889  49.555556   
            Scienze attuariali                      45051.500000  44.954545   
            Transaction Banking                     36616.555556  49.666667   
            Wealth & Asset Management               45496.941176  51.176471   

                                                    ID_ORGANIZATIONAL_POSITION  \
ID_PERSONAS DESCR_KNOWLEDGE_AREA                                                 
P00001      Advisory                                               7100.384615   
            Amministrazione                                        9003.055556   
            Analisi e Gestione del credito                         8560.357143   
            Analisi economica e finanziaria                        8461.230769   
            Architettura IT e software development                 6873.764706   
...                                                                        ...   
P00036      Risorse Umane                                         10886.807018   
            Safety                                                10891.666667   
            Scienze attuariali                                    10900.045455   
            Transaction Banking                                   10880.814815   
            Wealth & Asset Management                             10876.647059   

                                                      ID_MANAGER  \
ID_PERSONAS DESCR_KNOWLEDGE_AREA                                   
P00001      Advisory                                14231.000000   
            Amministrazione                         22833.722222   
            Analisi e Gestione del credito          16177.714286   
            Analisi economica e finanziaria         24535.076923   
            Architettura IT e software development  16346.117647   
...                                                          ...   
P00036      Risorse Umane                           59130.017544   
            Safety                                  49580.000000   
            Scienze attuariali                      59232.500000   
            Transaction Banking                     53441.851852   
            Wealth & Asset Management               51242.176471   

                                                    SCORE_AUTODETECTION  \
ID_PERSONAS DESCR_KNOWLEDGE_AREA                                          
P00001      Advisory                                           1.000000   
            Amministrazione                                    1.111111   
            Analisi e Gestione del credito                     1.000000   
            Analisi economica e finanziaria                    1.076923   
            Architettura IT e software development             1.000000   
...                                                                 ...   
P00036      Risorse Umane                                      1.140351   
            Safety                                             1.037037   
            Scienze attuariali                                 1.000000   
            Transaction Banking                                1.259259   
            Wealth & Asset Management                          1.470588   

                                                    SCORE_VALIDATION  
ID_PERSONAS DESCR_KNOWLEDG

### Create Network

In [56]:
df_nodes = pd.DataFrame(df["ID_EMPLOYEE"].unique())
df_nodes = df_nodes.rename(columns={0:"Node"})
df_nodes_domain = pd.DataFrame(df["DESCR_KNOWLEDGE_AREA"].unique())
df_nodes_domain = df_nodes_domain.rename(columns={0:"Node"})
df_nodes_domain = df_nodes_domain.dropna()
df_nodes = df_nodes.append(df_nodes_domain)

In [57]:
nodes = [n for n in df_nodes["Node"].to_numpy()][0:]

In [58]:
len(nodes)

9963

In [59]:
df_nodes

,Node
0,6
1,14705
2,7
3,15725
4,15730
...,...
38,Digital
39,Data governance
40,Credito e Finanza d'Impresa
41,Knowledge management


#### Create Edge

In [67]:
df_edges = df.groupby(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"]).mean()
df_edges

AGE  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                  
6           Amministrazione                                52.0   
            Analisi e Gestione del credito                 52.0   
            Business Development                           52.0   
            Capital Markets                                52.0   
            Conoscenza di Prodotti e Servizi Assicurativi  52.0   
...                                                         ...   
68366       Funding                                        49.0   
            Risorse Umane                                  49.0   
            Safety                                         49.0   
            Transaction Banking                            49.0   
            Wealth & Asset Management                      49.0   

                                                           ID_ORGANIZATIONAL_POSITION  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                                        
6           Amministrazione                                                      55.0   
            Analisi e Gestione del credito                                       55.0   
            Business Development                                                 55.0   
            Capital Markets                                                      55.0   
            Conoscenza di Prodotti e Servizi Assicurativi                        55.0   
...                                                                               ...   
68366       Funding                                                            8763.0   
            Risorse Umane                                                      8763.0   
            Safety                                                             8763.0   
            Transaction Banking                                                8763.0   
            Wealth & Asset Management                                          8763.0   

                                                           ID_MANAGER  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                        
6           Amministrazione                                   13080.0   
            Analisi e Gestione del credito                    13080.0   
            Business Development                              13080.0   
            Capital Markets                                   13080.0   
            Conoscenza di Prodotti e Servizi Assicurativi     13080.0   
...                                                               ...   
68366       Funding                                             760.0   
            Risorse Umane                                       760.0   
            Safety                                              760.0   
            Transaction Banking                                 760.0   
            Wealth & Asset Management                           760.0   

                                                           SCORE_AUTODETECTION  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                                 
6           Amministrazione                                                1.0   
            Analisi e Gestione del credito                                 1.0   
            Business Development                                           1.0   
            Capital Markets                                                4.0   
            Conoscenza di Prodotti e Servizi Assicurativi                  1.0   
...                                                                        ...   
68366       Funding                                                        1.0   
            Risorse Umane                                                  1.0   
            Safety                                                         1.0   
            Transaction Banking                                            1.0   
            Wealth & Asset Management                                      1.0   

             

In [68]:
df_edges = df_edges.reset_index(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"])
df_edges["SCORE_AVG"] = df_edges[["SCORE_AUTODETECTION","SCORE_VALIDATION"]].mean(axis=1)
df_edges = df_edges[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_AVG"]]
df_edges

,ID_EMPLOYEE,DESCR_KNOWLEDGE_AREA,SCORE_AVG
0,6,Amministrazione,1.00
1,6,Analisi e Gestione del credito,1.00
2,6,Business Development,1.00
3,6,Capital Markets,4.00
4,6,Conoscenza di Prodotti e Servizi Assicurativi,1.00
...,...,...,...
129459,68366,Funding,1.00
129460,68366,Risorse Umane,1.00
129461,68366,Safety,1.25
129462,68366,Transaction Banking,1.00


In [69]:
weighted_edges = [tuple(n) for n in df_edges.to_numpy()]

In [70]:
len(df_edges)

129464